<a href="https://colab.research.google.com/github/amitxsethi/Text2Sql-AIML/blob/main/gensql_group6_amit_llama7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Mon Nov 17 22:20:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             42W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

The `nbstripout` tool can help clean up notebook metadata, which might resolve the rendering issue. You'll need to specify the exact filename of this notebook. After running the command, you might need to refresh your browser page for the changes to take effect.

In [2]:
!git clone https://github.com/jieshi10/gensql.git
%cd gensql
!ls -la
print("✓ Repository cloned!")

Cloning into 'gensql'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 149 (delta 29), reused 148 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (149/149), 2.42 MiB | 19.85 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/gensql
total 132
drwxr-xr-x 10 root root  4096 Nov 17 22:20 .
drwxr-xr-x  1 root root  4096 Nov 17 22:20 ..
drwxr-xr-x  4 root root  4096 Nov 17 22:20 benchmark
drwxr-xr-x  3 root root  4096 Nov 17 22:20 bird_code
drwxr-xr-x  2 root root  4096 Nov 17 22:20 docker
drwxr-xr-x  8 root root  4096 Nov 17 22:20 .git
-rw-r--r--  1 root root  3139 Nov 17 22:20 .gitignore
-rw-r--r--  1 root root  1064 Nov 17 22:20 LICENSE
-rw-r--r--  1 root root 37953 Nov 17 22:20 main.py
drwxr-xr-x  2 root root  4096 Nov 17 22:20 pretrained
-rw-r--r--  1 root root 16684 Nov 17 22:20 process_sql.py
-rw-r--r--  1 root root  4259 Nov 17 22:20 README.md
-rw-r--r--  1 r

In [3]:
# Fix dependency issues first
!pip install -q jedi  # Fix ipython warning

# Install packages in the right order to avoid conflicts
print("Installing packages (this takes 2-3 minutes)...")

# Step 1: Core ML packages
!pip install -q torch torchvision torchaudio

# Step 2: NLP packages with compatible versions
!pip install -q 'transformers>=4.41.0,<5.0.0'
!pip install -q sentence-transformers
!pip install -q 'nltk>=3.9'  # Fix textblob conflict
!pip install -q openai

# Step 3: SQL and utility packages
!pip install -q 'sqlglot>=23.4'  # Use newer version
!pip install -q sqlparse
!pip install -q func-timeout
!pip install -q scikit-learn

# Step 4: Install vLLM last (takes longest)
print("\nInstalling vLLM (this takes 1-2 minutes)...")
!pip install -q vllm

print("\n✓ All packages installed successfully!")
print("\nInstalled versions:")
!pip list | grep -E 'torch|transformers|vllm|nltk|sqlglot'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.5 MB/s eta 0:00:00
Installing packages (this takes 2-3 minutes)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

Installing vLLM (this takes 1-2 minutes)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.2/438.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 140.4 MB/s eta 0:00

In [4]:
import nltk
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

print("✓ NLTK data downloaded!")

✓ NLTK data downloaded!


In [5]:
print("Fixing sentence_transformers compatibility...")

# Fix SentenceTransformer.py
with open('sentence_transformers/SentenceTransformer.py', 'r') as f:
    content = f.read()
content = content.replace(
    'from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, cached_download',
    'from huggingface_hub import HfApi, HfFolder, Repository, hf_hub_url, hf_hub_download'
)
with open('sentence_transformers/SentenceTransformer.py', 'w') as f:
    f.write(content)

# Fix util.py
with open('sentence_transformers/util.py', 'r') as f:
    content = f.read()

content = content.replace(
    'from huggingface_hub import HfApi, hf_hub_url, cached_download, HfFolder',
    '''from huggingface_hub import HfApi, hf_hub_url, hf_hub_download, HfFolder
try:
    from huggingface_hub import cached_download
except ImportError:
    cached_download = None'''
)

with open('sentence_transformers/util.py', 'w') as f:
    f.write(content)

print("✓ Fixed sentence_transformers!")

Fixing sentence_transformers compatibility...
✓ Fixed sentence_transformers!


In [6]:
from huggingface_hub import snapshot_download

print("Downloading BGE model (~1.3 GB)...")
print("This will take 2-3 minutes...\n")

model_path = snapshot_download(
    repo_id='BAAI/bge-large-en-v1.5',
    local_dir='./pretrained/bge-large-en-v1.5',
    local_dir_use_symlinks=False
)

print(f"\n✓ BGE model downloaded!")
!du -sh pretrained/bge-large-en-v1.5

This will take 2-3 minutes...



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]


✓ BGE model downloaded!
3.8G	pretrained/bge-large-en-v1.5


In [8]:
import os

os.makedirs('benchmark', exist_ok=True)

print("Downloading Spider dataset (~100 MB)...")
!wget -q --show-progress -O benchmark/spider.zip "https://drive.usercontent.google.com/download?id=1TqleXec_OykOYFREKKtschzY29dUcVAQ&export=download&confirm=t"

print("\nExtracting...")
!cd benchmark && unzip -q spider.zip && rm spider.zip

print("✓ Spider dataset ready!")
!du -sh benchmark/spider
!ls benchmark/spider/

benchmark/spider.zi 100%[===================>]  95.12M   261MB/s    in 0.4s    

Extracting...
replace spider/database/customer_deliveries/schema.sql? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace spider/database/customer_deliveries/customer_deliveries.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace spider/database/allergy_1/schema.sql? [y]es, [n]o, [A]ll, [N]one, [r]ename: ✓ Spider dataset ready!
878M	benchmark/spider
database      dev.json	  tables.json	  train_others.json
dev_gold.sql  README.txt  train_gold.sql  train_spider.json


In [9]:
from huggingface_hub import snapshot_download
import os

print("="*70)
print("DOWNLOADING CODELLAMA-7B-INSTRUCT")
print("="*70)
print("\nModel size: ~13 GB")
print("Time: 10-15 minutes\n")

model_path = snapshot_download(
    repo_id='codellama/CodeLlama-7b-Instruct-hf',
    local_dir='./models/CodeLlama-7b-Instruct',
    local_dir_use_symlinks=False
)

print(f"\n✓ CodeLlama-7B downloaded!")
print(f"Location: {model_path}")

# Check size
!du -sh ./models/CodeLlama-7b-Instruct

DOWNLOADING CODELLAMA-7B-INSTRUCT

Model size: ~13 GB
Time: 10-15 minutes



Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

USE_POLICY.md: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]


✓ CodeLlama-7B downloaded!
Location: /content/gensql/models/CodeLlama-7b-Instruct
26G	./models/CodeLlama-7b-Instruct


In [10]:
!lsof -ti :8000 && echo "⚠️  Port in use" || echo "✓ Port free"

✓ Port free


In [14]:
import subprocess
import time
import requests

# Kill any old processes
print("Stopping old vLLM processes...")
subprocess.run("pkill -9 -f vllm", shell=True)
time.sleep(3)
print("✓ Stopped\n")

print("="*70)
print("STARTING CODELLAMA-7B ON A100 (40GB)")
print("="*70)
print("\n⚙️ Settings:")
print("   • Model: ./models/CodeLlama-7b-Instruct")
print("   • Max context: 2048 tokens")
print("   • GPU memory: 0.85")
print("   • Max sequences: 16")
print("\n🕐 Loading model (60-90 seconds)...\n")

# Start vLLM with correct path
subprocess.Popen([
    'python', '-m', 'vllm.entrypoints.openai.api_server',
    '--host', '127.0.0.1',
    '--port', '8000',
    '--model', './models/CodeLlama-7b-Instruct',  # ← Correct path
    '--served-model-name', 'CodeLlama-7B-Instruct',
    '--max-model-len', '2048',
    '--gpu-memory-utilization', '0.85',
    '--dtype', 'float16',
    '--max-num-seqs', '16',
    '--disable-log-requests'
], stdout=open('vllm_codellama.log', 'w'), stderr=subprocess.STDOUT)

# Wait for server
print("Waiting for server to start...")
for i in range(15):
    time.sleep(10)
    try:
        r = requests.get('http://127.0.0.1:8000/health', timeout=2)
        if r.status_code == 200:
            models = requests.get('http://127.0.0.1:8000/v1/models').json()
            print(f"\n{'='*70}")
            print(f"✓ SERVER READY after {(i+1)*10} seconds!")
            print(f"✓ Model: {models['data'][0]['id']}")
            print('='*70)
            print("\n🎉 Ready to run Gen-SQL!\n")
            break
    except:
        print(f"⏳ {(i+1)*10}s...")
else:
    print("\n❌ Timeout. Checking logs:\n")
    subprocess.run("tail -30 vllm_codellama.log", shell=True)

Stopping old vLLM processes...
✓ Stopped

STARTING CODELLAMA-7B ON A100 (40GB)

⚙️ Settings:
   • Model: ./models/CodeLlama-7b-Instruct
   • Max context: 2048 tokens
   • GPU memory: 0.85
   • Max sequences: 16

🕐 Loading model (60-90 seconds)...

Waiting for server to start...
⏳ 10s...
⏳ 20s...
⏳ 30s...
⏳ 40s...
⏳ 50s...
⏳ 60s...
⏳ 70s...
⏳ 80s...
⏳ 90s...

✓ SERVER READY after 100 seconds!
✓ Model: CodeLlama-7B-Instruct

🎉 Ready to run Gen-SQL!



In [13]:
!tail -100 vllm_codellama.log

INFO 11-17 22:33:41 [__init__.py:216] Automatically detected platform cuda.
2025-11-17 22:33:42.221709: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 22:33:42.239048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763418822.259950    5233 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763418822.266350    5233 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763418822.282975    5233 computation_placer.cc:177] co

In [15]:
# Update the LLM client to use port 8000
with open('main.py', 'r') as f:
    code = f.read()

# Change port from 8080 to 8000
code = code.replace("port=8080", "port=8000")
code = code.replace("LLMClient(host='127.0.0.1', port=8080)",
                   "LLMClient(host='127.0.0.1', port=8000)")

with open('main.py', 'w') as f:
    f.write(code)

print("✓ Updated main.py to use port 9000")

✓ Updated main.py to use port 9000


In [16]:
!grep -n "DB_NAME\|FEW_SHOT_K\|MAX_SCHEMAS\|max_workers\|data\[:1\]" main.py | head -20

50:FEW_SHOT_K = 8
51:# FEW_SHOT_K = 4
52:# FEW_SHOT_K = 1
53:# FEW_SHOT_K = 0
58:MAX_SCHEMAS = 30
79:DB_NAME: str='bird'
80:# DB_NAME: str='spider'
84:if DB_NAME == 'bird':
94:elif DB_NAME == 'spider':
104:    raise ValueError(f'Unrecognized DB_NAME: {DB_NAME}')
113:        suf = f'fs{FEW_SHOT_K}'
118:        pre += f'-S{MAX_SCHEMAS}C{MAX_COLUMNS}-SS{SCHEMA_SCALE}CS{COLUMN_SCALE}-L{MAX_ITER}'
120:        pre += f'-S{MAX_SCHEMAS}'
328:    if DB_NAME == 'bird':
634:        examples = retriever.search_examples(question, FEW_SHOT_K)
638:                examples_by_sql = retriever.search_examples_by_sql(previous_sql, FEW_SHOT_K//2)
639:                examples_copied = sum(map(list, list(zip(examples[:FEW_SHOT_K//2], examples_by_sql))), [])
790:        .batch_search_schemas([search_query], MAX_SCHEMAS)[0]
860:            n = MAX_SCHEMAS
867:            n = min(n*SCHEMA_SCALE, MAX_SCHEMAS)


In [17]:
print("Configuring main.py for Colab...\n")

with open('main.py', 'r') as f:
    code = f.read()

# Change to Spider dataset
code = code.replace("DB_NAME: str='bird'", "DB_NAME: str='spider'")

# Optimize for Colab T4
code = code.replace("FEW_SHOT_K = 8", "FEW_SHOT_K = 8")
code = code.replace("MAX_SCHEMAS = 30", "MAX_SCHEMAS = 30")
code = code.replace("MAX_COLUMNS = 200", "MAX_COLUMNS = 200")
code = code.replace("MAX_ITER = 5", "MAX_ITER = 3")
code = code.replace("max_workers=50", "max_workers=20")

# Test on 1 example first
code = code.replace("# data = data[:20]", "data = data[:1]  # Test 1 example")

with open('main.py', 'w') as f:
    f.write(code)


Configuring main.py for Colab...



In [18]:
import nltk
import ssl

# Fix SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Download the new punkt_tab (required for NLTK 3.9+)
print("Downloading punkt_tab...")
nltk.download('punkt_tab', quiet=False)
nltk.download('averaged_perceptron_tagger_eng', quiet=False)

# Also download the old ones as backup
nltk.download('punkt', quiet=False)
nltk.download('averaged_perceptron_tagger', quiet=False)

print("\n✓ All NLTK data downloaded!")


✓ All NLTK data downloaded!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [19]:
# Create output directory
print("\n2. Creating output directory...")
import os
os.makedirs('output', exist_ok=True)
print("   ✓ Created")


2. Creating output directory...
   ✓ Created


In [20]:
print("="*70)
print("RUNNING GEN-SQL ON 1 EXAMPLE")
print("="*70)
print("\nThis will take 30-60 seconds...\n")

!python main.py

print("\n" + "="*70)
print("✓ TEST COMPLETED!")
print("="*70)

RUNNING GEN-SQL ON 1 EXAMPLE

This will take 30-60 seconds...

Use Model: CodeLlama-7B-Instruct
2025-11-17 22:53:15.956519: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 22:53:15.972161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763419995.993173   10593 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763419995.999529   10593 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763419996.015711   10593 computati

In [21]:
import json
import glob

output_files = glob.glob('output/*.json')

if output_files:
    print(f"Found output: {output_files[0]}\n")

    with open(output_files[0], 'r') as f:
        results = json.load(f)

    if len(results) > 0 and results[0] is not None:
        r = results[0]

        print("="*70)
        print("📊 RESULT FOR FIRST EXAMPLE")
        print("="*70)

        print(f"\n📝 Question:")
        print(f"   {r['question']}")

        print(f"\n💾 Database:")
        print(f"   {r['db']}")

        print(f"\n✅ Ground Truth SQL:")
        print(f"   {r['output']}")

        print(f"\n🤖 Generated SQL:")
        print(f"   {r['infer']}")

        # Check if they match
        match = r['output'].strip().lower() == r['infer'].strip().lower()
        if match:
            print("\n🎉 EXACT MATCH! Perfect generation!")
        else:
            print("\n⚠️  Different from ground truth")
            print("   (May still be semantically correct)")

        print(f"\n📈 Statistics:")
        print(f"   Rounds/Iterations: {len(r['rounds'])}")
        print(f"   Stop reason: {r['stop_reason']}")
        print(f"   Total time: {r['time']:.2f} seconds")

        print("\n" + "="*70)
        print("🔄 ROUND-BY-ROUND BREAKDOWN")
        print("="*70)
        for i, rd in enumerate(r['rounds']):
            print(f"\nRound {i}:")
            sql_preview = rd['infer'][:70] + "..." if len(rd['infer']) > 70 else rd['infer']
            print(f"   SQL: {sql_preview}")
            print(f"   Time: {rd['time']:.2f}s")
            if 'usage' in rd:
                usage = rd['usage']
                print(f"   Tokens: prompt={usage.get('prompt_tokens', 0)}, completion={usage.get('completion_tokens', 0)}")

        print("\n" + "="*70)
        print("✅ GEN-SQL IS WORKING PERFECTLY!")
        print("="*70)
    else:
        print("⚠️ No valid results in output file")
else:
    print("❌ No output files found!")

Found output: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L3-q_skltn-sql_skltn-sql_post_process-fs8-dev.json

📊 RESULT FOR FIRST EXAMPLE

📝 Question:
   How many singers do we have?

💾 Database:
   concert_singer

✅ Ground Truth SQL:
   SELECT count(*) FROM singer

🤖 Generated SQL:
   SELECT  count(*) FROM singer

⚠️  Different from ground truth
   (May still be semantically correct)

📈 Statistics:
   Rounds/Iterations: 2
   Stop reason: early_stopping
   Total time: 0.54 seconds

🔄 ROUND-BY-ROUND BREAKDOWN

Round 0:
   SQL: SELECT  count(*) FROM Singers
   Time: 0.19s
   Tokens: prompt=212, completion=7

Round 1:
   SQL: SELECT  count(*) FROM singer
   Time: 0.50s
   Tokens: prompt=640, completion=6

✅ GEN-SQL IS WORKING PERFECTLY!


In [22]:
print("Configuring for 200 examples...\n")

with open('main.py', 'r') as f:
    code = f.read()
code = code.replace("data = data[:1]  # Test 1 example", "data = data[:200]  # Test 200 examples")
with open('main.py', 'w') as f:
    f.write(code)

print("✓ Ready to run on 200 examples")
print("\nRun the next cell to execute:")

Configuring for 200 examples...

✓ Ready to run on 200 examples

Run the next cell to execute:


In [23]:
!python main.py

Use Model: CodeLlama-7B-Instruct
2025-11-17 22:55:25.254177: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 22:55:25.272614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763420125.294849   11208 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763420125.301707   11208 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763420125.318781   11208 computation_placer.cc:177] computation placer already registered. Please

In [24]:
import glob
import json

# Find the output file
output_files = glob.glob('output/*.json')
print(f"Output file: {output_files[0]}\n")

# Quick stats
with open(output_files[0], 'r') as f:
    results = json.load(f)

results = [r for r in results if r is not None]
print(f"Total examples processed: {len(results)}")
print(f"Avg time per example: {sum(r['time'] for r in results)/len(results):.2f}s")
print(f"Total time: {sum(r['time'] for r in results):.2f}s\n")

# Show first few questions
print("Questions processed:")
for i, r in enumerate(results[:5]):
    print(f"  {i+1}. {r['question'][:60]}...")

Output file: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L3-q_skltn-sql_skltn-sql_post_process-fs8-dev.json

Total examples processed: 147
Avg time per example: 6.64s
Total time: 976.11s

Questions processed:
  1. How many singers do we have?...
  2. What is the total number of singers?...
  3. Show name, country, age for all singers ordered by age from ...
  4. What are the names, countries, and ages for every singer in ...
  5. What is the average, minimum, and maximum age for all French...


In [32]:
%%writefile evaluate.py
#!/usr/bin/env python3
"""
Enhanced evaluation script for Gen-SQL with normalized EM and EX metrics
Loads ground truth from Spider dev.json
"""

import json
import sqlite3
import glob
import re


def normalize_sql(sql):
    """
    Normalize SQL for fair comparison:
    - Convert to lowercase
    - Remove extra whitespace
    - Remove trailing semicolons
    """
    if not sql:
        return ""

    # Convert to lowercase
    sql = sql.lower().strip()

    # Remove trailing semicolon
    sql = sql.rstrip(';').strip()

    # Normalize whitespace (multiple spaces to single space)
    sql = re.sub(r'\s+', ' ', sql)

    # Normalize spaces around operators and punctuation
    sql = re.sub(r'\s*,\s*', ', ', sql)  # Comma spacing
    sql = re.sub(r'\s*\(\s*', '(', sql)  # Opening paren
    sql = re.sub(r'\s*\)\s*', ')', sql)  # Closing paren

    return sql.strip()


def load_ground_truth(dev_file='benchmark/spider/dev.json'):
    """Load ground truth from Spider dev.json"""
    try:
        with open(dev_file, encoding='utf-8') as f:
            dev_data = json.load(f)

        # Create lookup: (question, db_id) -> gold_sql
        ground_truth = {}
        for item in dev_data:
            key = (item['question'].strip(), item['db_id'])
            ground_truth[key] = item['query']

        print(f"✓ Loaded {len(ground_truth)} ground truth examples from {dev_file}")
        return ground_truth
    except FileNotFoundError:
        print(f"❌ Ground truth file not found: {dev_file}")
        return {}


def calculate_exact_match(results, ground_truth):
    """Calculate normalized Exact Match accuracy"""
    if not results:
        print("❌ No valid results to evaluate!")
        return 0.0

    print(f"\n📊 EXACT MATCH (EM) - Normalized")
    print("="*70)

    matches = 0
    total = 0
    no_ground_truth = 0

    examples = []
    for i, result in enumerate(results, 1):
        question = result.get('question', '').strip()
        db_id = result.get('db', result.get('db_id', ''))
        pred_sql = normalize_sql(result.get('infer', ''))

        # Look up ground truth
        key = (question, db_id)
        if key not in ground_truth:
            no_ground_truth += 1
            continue

        gold_sql = normalize_sql(ground_truth[key])
        total += 1

        is_match = (pred_sql == gold_sql)
        if is_match:
            matches += 1

        # Store first 5 for display
        if len(examples) < 5:
            examples.append({
                'num': i,
                'question': question[:60] + '...',
                'gold': gold_sql[:70] + '...' if len(gold_sql) > 70 else gold_sql,
                'pred': pred_sql[:70] + '...' if len(pred_sql) > 70 else pred_sql,
                'match': '✓' if is_match else '✗'
            })

    accuracy = (matches / total * 100) if total > 0 else 0

    print(f"Matched: {matches}/{total}")
    print(f"Accuracy: {accuracy:.1f}%")
    if no_ground_truth > 0:
        print(f"⚠️  Skipped (no ground truth): {no_ground_truth}")

    # Show examples
    print("\n📋 First 5 examples:")
    print("-"*70)
    for ex in examples:
        print(f"\n{ex['num']}. {ex['question']}")
        print(f"   Gold: {ex['gold']}")
        print(f"   Pred: {ex['pred']}")
        print(f"   Match: {ex['match']}")

    return accuracy


def calculate_execution_accuracy(results, ground_truth):
    """Calculate Execution Accuracy (EX)"""
    if not results:
        print("❌ No valid results to evaluate!")
        return 0.0, []

    print(f"\n📊 EXECUTION ACCURACY (EX)")
    print("="*70)

    correct = 0
    total = 0
    errors = []
    db_not_found = 0
    no_ground_truth = 0

    for i, result in enumerate(results, 1):
        question = result.get('question', '').strip()
        db_id = result.get('db', result.get('db_id', ''))

        # Look up ground truth
        key = (question, db_id)
        if key not in ground_truth:
            no_ground_truth += 1
            continue

        gold_sql = ground_truth[key]
        db_path = f"benchmark/spider/database/{db_id}/{db_id}.sqlite"

        try:
            # Check if database exists
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            # Execute predicted SQL
            pred_sql = result.get('infer', '').strip()
            cursor.execute(pred_sql)
            pred_results = cursor.fetchall()

            # Execute gold SQL
            cursor.execute(gold_sql)
            gold_results = cursor.fetchall()

            total += 1

            # Compare results
            if pred_results == gold_results:
                correct += 1
            else:
                errors.append({
                    'example': i,
                    'question': question,
                    'db': db_id,
                    'reason': 'Results mismatch',
                    'pred_rows': len(pred_results),
                    'gold_rows': len(gold_results)
                })

            conn.close()

        except sqlite3.Error as e:
            total += 1
            errors.append({
                'example': i,
                'question': question,
                'db': db_id,
                'error': str(e)[:100]
            })
        except FileNotFoundError:
            db_not_found += 1
            errors.append({
                'example': i,
                'question': question,
                'db': db_id,
                'error': 'Database file not found'
            })

    accuracy = (correct / total * 100) if total > 0 else 0

    print(f"Correct: {correct}/{total}")
    print(f"Accuracy: {accuracy:.1f}%")
    print(f"Execution errors: {len(errors)}")
    if db_not_found > 0:
        print(f"Database not found: {db_not_found}")
    if no_ground_truth > 0:
        print(f"⚠️  Skipped (no ground truth): {no_ground_truth}")

    return accuracy, errors


def main():
    """Main evaluation pipeline"""
    print("="*70)
    print("GEN-SQL EVALUATION: Normalized EM + EX")
    print("="*70)

    # Find latest output (exclude _report and _errors files)
    pattern = 'output/*-spider-*.json'
    files = [f for f in glob.glob(pattern)
             if '_report' not in f and '_errors' not in f]

    if not files:
        print(f"❌ No results found matching: {pattern}")
        return

    results_file = files[0]
    print(f"\n📂 Results: {results_file}")

    # Load results
    with open(results_file, encoding='utf-8') as f:
        all_results = json.load(f)
        valid_results = [r for r in all_results if r and isinstance(r, dict)]
        null_entries = len(all_results) - len(valid_results)

    print(f"\nValid results: {len(valid_results)}")
    print(f"Null entries: {null_entries}")
    print(f"Total: {len(all_results)}")

    if not valid_results:
        print("\n❌ No valid results to evaluate!")
        return

    # Load ground truth
    ground_truth = load_ground_truth()
    if not ground_truth:
        print("❌ Cannot evaluate without ground truth!")
        return

    print(f"\nEvaluating {len(valid_results)} examples...")

    # Calculate both metrics
    em_accuracy = calculate_exact_match(valid_results, ground_truth)
    ex_accuracy, errors = calculate_execution_accuracy(valid_results, ground_truth)

    # Summary
    print("\n" + "="*70)
    print("FINAL RESULTS")
    print("="*70)
    print(f"Examples evaluated:      {len(valid_results)}")
    print(f"Null/Failed:             {null_entries}")
    print()
    print(f"Exact Match (EM):        {em_accuracy:.1f}% (normalized)")
    print(f"Execution Accuracy (EX): {ex_accuracy:.1f}%")
    print(f"Execution errors:        {len(errors)}")
    print()
    if ex_accuracy > em_accuracy:
        print(f"✓ EX better than EM: +{ex_accuracy - em_accuracy:.1f}%")
        print(f"  (Semantically correct despite syntax differences)")
    elif ex_accuracy < em_accuracy:
        print(f"⚠️  EX lower than EM: {ex_accuracy - em_accuracy:.1f}%")
        print(f"  (Some exact matches don't execute correctly)")
    else:
        print("ℹ️  EM and EX are equal")
    print("="*70)

    # Save errors for analysis
    if errors:
        error_file = results_file.replace('.json', '_errors.json')
        with open(error_file, 'w', encoding='utf-8') as f:
            json.dump(errors, f, indent=2)
        print(f"\n💾 Errors saved to: {error_file}")


if __name__ == '__main__':
    main()

Overwriting evaluate.py


In [33]:
!python evaluate.py

GEN-SQL EVALUATION: Normalized EM + EX

📂 Results: output/CodeLlama-7B-Instruct-spider-gtrr_column-S30C200-SS4CS6-L3-q_skltn-sql_skltn-sql_post_process-fs8-dev.json

Valid results: 147
Null entries: 53
Total: 200
✓ Loaded 1034 ground truth examples from benchmark/spider/dev.json

Evaluating 147 examples...

📊 EXACT MATCH (EM) - Normalized
Matched: 56/147
Accuracy: 38.1%

📋 First 5 examples:
----------------------------------------------------------------------

1. How many singers do we have?...
   Gold: select count(*)from singer
   Pred: select count(*)from singer
   Match: ✓

2. What is the total number of singers?...
   Gold: select count(*)from singer
   Pred: select count(*)from singer
   Match: ✓

3. Show name, country, age for all singers ordered by age from ...
   Gold: select name, country, age from singer order by age desc
   Pred: select name, country, age from singer order by age desc
   Match: ✓

4. What are the names, countries, and ages for every singer in ...
   Gold: 